In [1]:
import pandas as pd
import numpy as np

from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):

    R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km
    R = 6372.8 #kms, ignores variations between pole and equation, irrelevant for short distances

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

def haversine_vec(s_lat, s_lng, e_lat, e_lng):

    # approximate radius of earth in km
    R = 6372.8

    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

    return 2 * R * np.arcsin(np.sqrt(d))


In [ ]:
# You need geocoder data in "geocoder" and put the DFG's project data in a folder called "refreshed_data"

In [3]:
# Loading normal zipcodes of houses
df_zipcodes = pd.read_csv('./geocoder/zipcodeset.txt')
df_house = pd.read_csv('./refreshed_data/Data/Location.txt',sep='|')
df_members = pd.read_csv('./refreshed_data/Data/Member.txt',sep='|')
# print(df_members.shape)
# df_members = df_members[df_members.PostalCode != 'NaN' ]
df_members = df_members.dropna(subset=['PostalCode'])
# print(df_members.shape)

df_members.head()
# df_zipcodes.head()
# df_house_zips.head(20)

,D4G_MemberId,MemberNumber,MemType,Active,MemberLocation,City,Province,PostalCode,D4G_ValidPostalCode,BirthYear,...,MemberFeeOutstanding,MemberFeesTotal,Suspended,Camper,CampYear,MemberIdentity,Subsidy,HearAbout,FamilyIdentity,Age
0,-2147249760,4fd535de3a32f6982a56bb78773e11e0,M,Y,CAMP,Nepean,ON,K2L 4W4,NaN,2004.0,...,0.0,0.0,N,N,1900-01-01,21708.0,N,NONE,NaN,14.0
1,-2147244363,ef6696776594bfb03ad29f41838a704a,X,N,BRIT,Ottawa,ON,K2B 8A3,K2B 8A,1990.0,...,0.0,25.0,N,NaN,NaN,1809.0,N,NONE,0,29.0
2,-2147207492,ca70ca5f6efd8e48f2b4b75b3bd75a5f,X,N,PYC,Ottawa,ON,K1Z 5A7,K1Z 5A,1996.0,...,0.0,0.0,Y,N,1900-01-01,12483.0,N,NONE,0,23.0
3,-2146927475,d74a123791c30d807f7cb512416368b3,M,N,HEA,Ottawa,ON,K1V 6A4,NaN,2000.0,...,0.0,0.0,N,N,1900-01-01,14619.0,N,NONE,NaN,19.0
6,-2145930611,7369e83e5d82fb2bbaf1745f83326ff6,V,N,ADM,Ottawa,ON,K2B7W1,K2B 7W,1978.0,...,NaN,NaN,NaN,N,NaN,18528.0,NaN,NaN,NaN,41.0


In [4]:
df_members_zipcodes = pd.DataFrame(df_members[['D4G_MemberId','PostalCode']],columns=['D4G_MemberId','PostalCode'])
df_members_zipcodes.head()


,D4G_MemberId,PostalCode
0,-2147249760,K2L 4W4
1,-2147244363,K2B 8A3
2,-2147207492,K1Z 5A7
3,-2146927475,K1V 6A4
6,-2145930611,K2B7W1


In [5]:
# Let's figure out what's happening with the zipcodes

def clean_up_zip(x,df_zipcodes):
    zipcode = x
    zipcode = zipcode.replace(" ", "")
    zipcode = zipcode.replace('-','')
    zipcode = zipcode.upper()
    if len(zipcode) == 6 :
        return zipcode
    
    else:
        return 'Less than 6'

      
df_members_zipcodes['PostalCodeClean'] = df_members_zipcodes.PostalCode.apply(clean_up_zip, args=[df_zipcodes])

df_members_gis = pd.merge(df_members_zipcodes,
                          df_zipcodes, 
                          left_on='PostalCodeClean',
                          right_on='PostCode',
                          how='left')


df_members_gis.head()
# df_members_gis.shape
# df_members_zipcodes

,D4G_MemberId,PostalCode,PostalCodeClean,PostCode,Latitude,Longitude,City,Province,CityAlt,Neighborhood,Time Zone,Name,Area Code
0,-2147249760,K2L 4W4,K2L4W4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2147244363,K2B 8A3,K2B8A3,K2B8A3,45.348819,-75.795493,Ottawa,ON,Ottawa,Whitehaven,UTC-05:00,America/Toronto,613
2,-2147207492,K1Z 5A7,K1Z5A7,K1Z5A7,45.373429,-75.732654,Ottawa,ON,Ottawa,Carlington,UTC-05:00,America/Toronto,613
3,-2146927475,K1V 6A4,K1V6A4,K1V6A4,45.378797,-75.653068,Ottawa,ON,Ottawa,South Keys,UTC-05:00,America/Toronto,613
4,-2145930611,K2B7W1,K2B7W1,K2B7W1,45.355241,-75.785250,Ottawa,ON,Ottawa,Whitehaven,UTC-05:00,America/Toronto,613


In [6]:
# we drop the ones that are not valid
print('Users have no valid zipcode:', df_members_gis.PostCode.isnull().sum())
df_members_gis = df_members_gis.dropna(axis=0,subset=['PostCode'])
df_members_gis.head()
# df_members_gis.shape

Users have no valid zipcode: 1228


,D4G_MemberId,PostalCode,PostalCodeClean,PostCode,Latitude,Longitude,City,Province,CityAlt,Neighborhood,Time Zone,Name,Area Code
1,-2147244363,K2B 8A3,K2B8A3,K2B8A3,45.348819,-75.795493,Ottawa,ON,Ottawa,Whitehaven,UTC-05:00,America/Toronto,613
2,-2147207492,K1Z 5A7,K1Z5A7,K1Z5A7,45.373429,-75.732654,Ottawa,ON,Ottawa,Carlington,UTC-05:00,America/Toronto,613
3,-2146927475,K1V 6A4,K1V6A4,K1V6A4,45.378797,-75.653068,Ottawa,ON,Ottawa,South Keys,UTC-05:00,America/Toronto,613
4,-2145930611,K2B7W1,K2B7W1,K2B7W1,45.355241,-75.785250,Ottawa,ON,Ottawa,Whitehaven,UTC-05:00,America/Toronto,613
5,-2145862491,K1V 8Z5,K1V8Z5,K1V8Z5,45.372368,-75.649241,Ottawa,ON,Ottawa,South Keys,UTC-05:00,America/Toronto,613


In [7]:
# we now calculate all the stuff 
members_coords = df_members_gis[['Latitude','Longitude']].values

for index,row in df_house.iterrows():
#     print(row['Location'])
    distances = haversine_vec(members_coords[:,0],members_coords[:,1],
                          row['Lat'],row['Lon'])
    df_members_gis = df_members_gis.join(pd.DataFrame(distances, columns=['dist_to_' + row['Location']]))



In [8]:
df_members_gis.head()

,D4G_MemberId,PostalCode,PostalCodeClean,PostCode,Latitude,Longitude,City,Province,CityAlt,Neighborhood,...,dist_to_HGT,dist_to_RID,dist_to_CAMP,dist_to_RGM,dist_to_HEA,dist_to_BAY,dist_to_PWH,dist_to_BRC,dist_to_ROC,dist_to_PAL
1,-2147244363,K2B 8A3,K2B8A3,K2B8A3,45.348819,-75.795493,Ottawa,ON,Ottawa,Whitehaven,...,6.178596,9.782259,104.333907,5.772216,6.518367,6.918784,1.734164,6.918784,4.389326,8.842469
2,-2147207492,K1Z 5A7,K1Z5A7,K1Z5A7,45.373429,-75.732654,Ottawa,ON,Ottawa,Carlington,...,0.253328,6.309107,110.317620,0.532953,0.774046,12.983910,5.699126,12.983910,5.202129,5.565227
3,-2146927475,K1V 6A4,K1V6A4,K1V6A4,45.378797,-75.653068,Ottawa,ON,Ottawa,South Keys,...,10.549883,14.203347,100.755533,10.148437,10.798180,2.342926,5.050774,2.342926,8.564414,13.274927
4,-2145930611,K2B7W1,K2B7W1,K2B7W1,45.355241,-75.785250,Ottawa,ON,Ottawa,Whitehaven,...,0.582411,6.993558,110.746741,0.871577,0.001051,13.103711,5.770139,13.103711,5.892906,6.281174
5,-2145862491,K1V 8Z5,K1V8Z5,K1V8Z5,45.372368,-75.649241,Ottawa,ON,Ottawa,South Keys,...,7.200410,1.900089,108.212513,7.185199,7.714537,15.453210,9.921267,15.453210,4.745837,1.761016


In [10]:
df_members_gis.shape


(22756, 29)

In [ ]:
# Save if you want!
df_members_gis.to_csv('members_dist_to_houses.csv')